In [1]:
import cv2
import os
import re
import pandas as pd
from datetime import datetime
from ultralytics import YOLO
from skimage import exposure
import numpy as np
from typing import List, Tuple
from concurrent.futures import ThreadPoolExecutor
import logging

logging.getLogger("ultralytics").setLevel(logging.ERROR)

# Constantes
WINDOW_SIZE = 254
ROOT_FOLDER = "/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms"
CSV_PATH = os.path.join(ROOT_FOLDER, "model_comparison.csv")
PATTERN = re.compile(r"paleta-(\d+)-(\d+)\.(png|jpg|jpeg)", re.IGNORECASE)

# Configurações
RUN_SINGLE_FOLDER_ONLY = False
ESPECIFIC_FOLDER = "base-4"

model_paths = [
    "/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/yolo-train/eggs-scanner-image.v2i.yolov11/runs/detect/train2/weights/best-train2.onnx",
]

class ImageProcessor:
    def __init__(self, model: YOLO, window_size: int = WINDOW_SIZE) -> None:
        self.window_size = window_size
        self.model = model
        self.image = None
        self.processed_image = None
        self.processed_windows = []
        self.object_counts = []

    def load_image(self, image_path: str) -> np.ndarray:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
        return image

    def normalize_square(self, square: np.ndarray) -> np.ndarray:
        return exposure.adjust_gamma(square, gamma=1.5)

    def predict_and_draw_boxes(self, square: np.ndarray) -> Tuple[int, np.ndarray]:
        """Executa a inferência e desenha as caixas detectadas com label 'ovo XX.XX (%)' em vermelho."""
        results = self.model(square, verbose=False)
        boxes = results[0].boxes
        output = square.copy()

        if boxes is None or boxes.xyxy is None:
            return 0, output

        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            score = float(box.conf[0]) if box.conf is not None else 0.0
            label = f"ovo {score * 100:.2f} (%)"

            # Caixa vermelha
            cv2.rectangle(output, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # Texto acima da caixa
            text_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            text_x = x1
            text_y = y1 - 5 if y1 - 5 > 10 else y1 + text_size[1] + 5
            cv2.putText(output, label, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        return len(boxes), output

    def process_single_window(self, window: np.ndarray, y: int, x: int, index: int) -> Tuple[int, np.ndarray, int, np.ndarray, int, int]:
        processed_window = self.normalize_square(window)
        object_count, counted_image = self.predict_and_draw_boxes(processed_window.copy())
        return index, processed_window, object_count, counted_image, y, x

    def apply_window_pipeline(self) -> None:
        if self.image is None:
            raise ValueError("Nenhuma imagem foi carregada para processamento.")

        img_height, img_width, channels = self.image.shape
        padded_height = ((img_height + self.window_size - 1) // self.window_size) * self.window_size
        padded_width = ((img_width + self.window_size - 1) // self.window_size) * self.window_size
        padded_image = np.zeros((padded_height, padded_width, channels), dtype=self.image.dtype)
        padded_image[:img_height, :img_width, :] = self.image
        processed_image = np.copy(padded_image)

        self.processed_windows.clear()
        self.object_counts.clear()

        tasks = []
        with ThreadPoolExecutor() as executor:
            for y in range(0, padded_height, self.window_size):
                for x in range(0, padded_width, self.window_size):
                    window = padded_image[y:y + self.window_size, x:x + self.window_size]
                    future = executor.submit(self.process_single_window, window, y, x, len(tasks))
                    tasks.append(future)

        results = sorted((future.result() for future in tasks), key=lambda r: r[0])
        for res in results:
            idx, processed_window, object_count, counted_image, y, x = res
            self.processed_windows.append(processed_window)
            self.object_counts.append(object_count)
            processed_image[y:y + self.window_size, x:x + self.window_size] = counted_image

        self.processed_image = processed_image[:img_height, :img_width]

    def process_folder(self, folder_path: str, model_name: str) -> List[Tuple[str, int, int, float]]:
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        start_time = datetime.now()
        results = []

        image_files.sort()

        for i, filename in enumerate(image_files):
            full_path = os.path.join(folder_path, filename)
            match = PATTERN.match(filename)
            real_eggs = int(match.group(2)) if match else -1

            try:
                self.image = self.load_image(full_path)
            except ValueError as e:
                print(e)
                continue

            process_start_time = datetime.now()
            self.apply_window_pipeline()
            elapsed_time = (datetime.now() - process_start_time).total_seconds()

            total_objects = sum(self.object_counts)
            results.append((filename, real_eggs, total_objects, elapsed_time))

            if i < 2:
                processed_output_folder = os.path.join(folder_path, "processed")
                os.makedirs(processed_output_folder, exist_ok=True)
                output_image_path = os.path.join(processed_output_folder, f"processed_{model_name}_{filename}")
                cv2.imwrite(output_image_path, self.processed_image)  # ✅ corrigido aqui

        elapsed_total_time = (datetime.now() - start_time).total_seconds()
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        report_path = os.path.join(folder_path, f"report-{timestamp}-{model_name}.txt")

        with open(report_path, "w") as f:
            f.write("+===================================+\n")
            f.write("Filename - Ovos reais - Ovos contados\n")
            f.write(f"Modelo: {model_name}\n")
            f.write("+===================================+\n")
            for filename, real_eggs, counted_eggs, _ in results:
                f.write(f"{filename} - {real_eggs} - {counted_eggs}\n")
            f.write("+===================================+\n")
            f.write(f"Tempo total de execução: {elapsed_total_time:.2f}s\n")

        return results

def find_folders(root_path: str) -> List[str]:
    if RUN_SINGLE_FOLDER_ONLY and ESPECIFIC_FOLDER:
        folder_path = os.path.join(root_path, ESPECIFIC_FOLDER)
        return [folder_path] if os.path.isdir(folder_path) else []

    folders = []
    for d in sorted(os.listdir(root_path)):
        if d.startswith("base-"):
            match = re.match(r"base-(\d+)", d)
            if match:
                index = int(match.group(1))
                if 4 <= index <= 12:
                    folders.append(os.path.join(root_path, d))
    return folders

def update_csv(folder: str, results: List[Tuple[str, int, int, float]], model_name: str) -> None:
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data = pd.DataFrame({
        "name_file": [folder] * len(results),
        "real_eggs_images": [result[1] for result in results],
        "counted_eggs_image": [result[2] for result in results],
        "model_name": [model_name] * len(results),
        "total_time": [result[3] for result in results],
        "image_name": [result[0] for result in results],
        "timestamp": [timestamp] * len(results)
    })

    if os.path.exists(CSV_PATH):
        data.to_csv(CSV_PATH, mode='a', header=False, index=False)
    else:
        data.to_csv(CSV_PATH, mode='w', header=True, index=False)

def load_model_safe(model_path: str) -> YOLO:
    try:
        model = YOLO(model_path)
        dummy = np.zeros((256, 256, 3), dtype=np.uint8)
        model(dummy)
        return model
    except Exception as e:
        if model_path.endswith(".onnx"):
            print(f"⚠️ Erro ao carregar modelo ONNX: {e}")
            fallback_path = model_path.replace(".onnx", ".pt")
            print(f"🔁 Tentando carregar fallback: {fallback_path}")
            return YOLO(fallback_path)
        else:
            raise e

def main():
    folders = find_folders(ROOT_FOLDER)
    print(f"📂 Pastas encontradas: {len(folders)}")

    for model_path in model_paths:
        yolo = load_model_safe(model_path)
        model_name = os.path.basename(model_path)
        print(f"\n🔍 Processando com modelo: {model_name}")
        processor = ImageProcessor(yolo)

        for folder in folders:
            print(f"📂 Pasta: {folder}")
            results = processor.process_folder(folder, model_name)
            update_csv(folder, results, model_name)

            for filename, real_eggs, total_counted, elapsed_time in results:
                print(f"🖼️  Imagem: {filename} | Real: {real_eggs} | Contado: {total_counted} | Tempo: {elapsed_time:.2f}s")

if __name__ == "__main__":
    main()


📂 Pastas encontradas: 9


2025-04-12 13:14:53.573279566 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.



🔍 Processando com modelo: best-train2.onnx
📂 Pasta: /media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/base-10
🖼️  Imagem: paleta-10-128.jpg | Real: 128 | Contado: 136 | Tempo: 4.39s
🖼️  Imagem: paleta-11-12.jpg | Real: 12 | Contado: 15 | Tempo: 3.48s
🖼️  Imagem: paleta-11-169.jpg | Real: 169 | Contado: 159 | Tempo: 3.44s
🖼️  Imagem: paleta-12-10.jpg | Real: 10 | Contado: 17 | Tempo: 3.35s
🖼️  Imagem: paleta-12-34.jpg | Real: 34 | Contado: 32 | Tempo: 3.11s
🖼️  Imagem: paleta-14-89.jpg | Real: 89 | Contado: 88 | Tempo: 3.50s
🖼️  Imagem: paleta-15-111.jpg | Real: 111 | Contado: 58 | Tempo: 3.49s
🖼️  Imagem: paleta-16-172.jpg | Real: 172 | Contado: 225 | Tempo: 3.07s
🖼️  Imagem: paleta-17-54.jpg | Real: 54 | Contado: 54 | Tempo: 3.05s
🖼️  Imagem: paleta-17-90.jpg | Real: 90 | Contado: 107 | Tempo: 2.83s
🖼️  Imagem: paleta-3-56.jpg | Real: 56 | Contado: 54 | Tempo: 2.93s
🖼️  Imagem: paleta-4-34.jpg | Real: 34 | Contado: 54 | Tempo: 3.12s
🖼️  Imagem: paleta-5-11.jpg | Real: 